# 과업 2: 수상 레저 속 번호판 탐지 및 텍스트 추출 (OCR)

## 개요
이 노트북은 학습된 번호판 탐지 모델과 EasyOCR을 사용하여 수상 레저 이미지에서 번호판을 탐지하고 텍스트를 추출합니다.

## 주요 기능
- **번호판 탐지**: 학습된 YOLO 모델로 번호판 영역 탐지
- **OCR 텍스트 추출**: EasyOCR을 사용하여 번호판에서 텍스트 추출
- **번호판 형식 정규화**: AA-00-0000 또는 00-AA-0000 형식으로 정규화
- **전처리 기법**: 회전, 크기 조정, 감마 보정 등 다양한 전처리 적용
- **결과 저장**: CSV 파일과 시각화 이미지로 결과 저장

## 입력 데이터
- `data/test/plate_detection/` 폴더의 테스트 이미지 파일들
- `model/plate_detection_baseline.pt` - 학습된 번호판 탐지 모델

## 출력
- `runs/ocr/OCR_FINAL_1029/` 폴더에 결과 저장
  - `ocr_results.csv`: OCR 결과 CSV 파일
  - `crops/`: 탐지된 번호판 크롭 이미지
  - `overlay/`: 원본 이미지에 탐지 결과를 오버레이한 이미지
  - `run_log.txt`: 실행 로그


In [ ]:
# 필요한 라이브러리 import
import sys
from pathlib import Path
import torch

# 프로젝트 루트 경로 설정
BASE_DIR = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.insert(0, str(BASE_DIR / "src"))

print(f"프로젝트 루트: {BASE_DIR}")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


## 1. 설정 및 경로 확인

먼저 필요한 경로와 파일이 존재하는지 확인합니다.


In [ ]:
# 경로 설정
WEIGHT_PATH = BASE_DIR / "model" / "plate_detection_baseline.pt"
SOURCE_DIR = BASE_DIR / "data" / "test" / "plate_detection"

print(f"[설정] 모델 경로: {WEIGHT_PATH}")
print(f"[설정] 테스트 이미지 경로: {SOURCE_DIR}")
print()

# 파일 존재 확인
if not WEIGHT_PATH.exists():
    print(f"[ERROR] 모델 파일이 없습니다: {WEIGHT_PATH}")
    print("[INFO] 먼저 plate_detection_train.py를 실행하여 모델을 학습하세요.")
else:
    print(f"[OK] 모델 파일 확인: {WEIGHT_PATH}")

if not SOURCE_DIR.exists():
    print(f"[ERROR] 테스트 이미지 폴더가 없습니다: {SOURCE_DIR}")
else:
    image_count = len(list(SOURCE_DIR.glob("*.jpg"))) + len(list(SOURCE_DIR.glob("*.png")))
    print(f"[OK] 테스트 이미지 폴더 확인: {SOURCE_DIR} ({image_count}개 이미지)")


## 2. OCR 추론 실행

`plate_ocr_inference.py` 스크립트를 실행하여 번호판 탐지 및 OCR을 수행합니다.


In [ ]:
# plate_ocr_inference.py 스크립트 실행
# 주의: 이 셀은 실행에 시간이 걸릴 수 있습니다 (이미지 수에 따라)

import subprocess
import os

# 작업 디렉토리를 프로젝트 루트로 변경
script_path = BASE_DIR / "src" / "plate_ocr_inference.py"

if script_path.exists():
    print(f"[INFO] 스크립트 실행 시작: {script_path}")
    print("[INFO] 이 과정은 시간이 걸릴 수 있습니다...")
    print()
    
    # Python 스크립트 실행
    result = subprocess.run(
        [sys.executable, str(script_path)],
        cwd=str(BASE_DIR),
        capture_output=False,
        text=True
    )
    
    if result.returncode == 0:
        print("\n[완료] OCR 추론이 성공적으로 완료되었습니다!")
    else:
        print(f"\n[오류] 스크립트 실행 중 오류가 발생했습니다. (반환 코드: {result.returncode})")
else:
    print(f"[ERROR] 스크립트 파일을 찾을 수 없습니다: {script_path}")


## 3. 결과 확인

OCR 결과를 확인합니다.


In [ ]:
# OCR 결과 CSV 파일 읽기
import pandas as pd
import matplotlib.pyplot as plt
import cv2

OUT_ROOT = BASE_DIR / "runs" / "ocr" / "OCR_FINAL_1029"
CSV_PATH = OUT_ROOT / "ocr_results.csv"
OVERLAY_DIR = OUT_ROOT / "overlay"

if CSV_PATH.exists():
    # CSV 파일 읽기
    df = pd.read_csv(CSV_PATH)
    print(f"[INFO] OCR 결과: {len(df)}개 탐지")
    print()
    print("=== OCR 결과 요약 ===")
    print(f"총 이미지: {df['image'].nunique()}개")
    print(f"총 탐지: {len(df)}개")
    print(f"유효한 번호판: {df['is_valid'].sum()}개")
    print()
    print("=== 상위 10개 결과 ===")
    print(df[['image', 'ocr_text', 'norm_text', 'is_valid']].head(10).to_string(index=False))
    
    # 유효한 번호판만 필터링
    valid_df = df[df['is_valid'] == 1]
    if len(valid_df) > 0:
        print()
        print("=== 유효한 번호판 ===")
        print(valid_df[['image', 'norm_text', 'pattern_type']].to_string(index=False))
else:
    print(f"[WARN] 결과 CSV 파일이 없습니다: {CSV_PATH}")


## 4. 결과 이미지 시각화

처리된 이미지 중 하나를 확인해봅시다.


In [ ]:
# 결과 이미지 확인
if OVERLAY_DIR.exists():
    overlay_files = list(OVERLAY_DIR.glob("*.jpg")) + list(OVERLAY_DIR.glob("*.png"))
    if overlay_files:
        # 첫 번째 결과 이미지 표시
        result_path = overlay_files[0]
        result_img = cv2.imread(str(result_path))
        if result_img is not None:
            result_img_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(14, 10))
            plt.imshow(result_img_rgb)
            plt.title(f"OCR 결과: {result_path.name}")
            plt.axis('off')
            plt.tight_layout()
            plt.show()
            
            # 해당 이미지의 OCR 결과 출력
            if CSV_PATH.exists():
                img_name = result_path.stem.replace("_overlay", "")
                img_results = df[df['image'].str.contains(img_name, na=False)]
                if len(img_results) > 0:
                    print(f"\n=== {img_name}의 OCR 결과 ===")
                    for idx, row in img_results.iterrows():
                        print(f"  탐지 {idx+1}:")
                        print(f"    원본 텍스트: {row['ocr_text']}")
                        print(f"    정규화 텍스트: {row['norm_text']}")
                        print(f"    패턴 타입: {row['pattern_type']}")
                        print(f"    유효성: {'유효' if row['is_valid'] else '무효'}")
                        print()
        else:
            print("이미지를 로드할 수 없습니다.")
    else:
        print("결과 이미지가 없습니다.")
else:
    print(f"결과 폴더가 없습니다: {OVERLAY_DIR}")
